In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import backend as K
import numpy as np
import os
import time
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session


config = tf.ConfigProto()
config.gpu_options.allow_growth = True  #dynamically grow the memory used on the GPU
set_session(tf.Session(config=config))

# dimensions of our images.
img_width, img_height = 512, 512

train_data_dir = 'E:\\UAV_drone\\data\\training\\vertical_view\\train'
validation_data_dir = 'E:\\UAV_drone\\data\\training\\vertical_view\\val'
nb_train_samples = 50041
nb_validation_samples = 12117
epochs = 100
batch_size = 8
img_channels = 3
nb_classes = 3

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(rescale=1. / 255)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(512, 512),
    batch_size=batch_size,
    classes=['down', 'medium', 'up'])

label_map = (train_generator.class_indices)
print(label_map)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(512, 512),
    batch_size=batch_size,
    classes=['down', 'medium', 'up'])


# checkpoint
current_directory = os.getcwd()

checkpoint_dir = os.path.join(current_directory, 'checkpoints')
result_dir = os.path.join(current_directory, 'results')

if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

Using TensorFlow backend.


Found 50041 images belonging to 3 classes.
{'down': 0, 'medium': 1, 'up': 2}
Found 12117 images belonging to 3 classes.


### Load base model

In [3]:
from keras.models import load_model    

model_file = 'model_resnet18_view-offset-altitude_12.hdf5'
model_full_path = os.path.join(checkpoint_dir, model_file)
model = load_model(model_full_path)

model.summary()

# weights = model.get_weights()
# print(len(weights))
# print(len(model.layers))
# print(len(weights[:-4]))
# for i in range(0, len(weights)):
#     print(weights[i].shape)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 256, 256, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

### Transfering to new model

In [4]:
from keras.models import Model
new_model = Model(model.inputs, model.layers[-3].output)

for i in range(0, (len(new_model.layers)-2)):
    new_model.layers[i].trainable = False
    
new_model.summary()
new_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

weights = model.get_weights()
new_model.set_weights(weights[:-4])

# delete(model)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 256, 256, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [5]:
#check if weights are loaded into the new_model
new_model.get_weights()

[array([[[[ 4.73237038e-03,  1.04166754e-01, -2.93186041e-32, ...,
           -2.93025116e-32, -7.55046634e-03,  2.90121939e-32],
          [-5.74592911e-02,  1.48635358e-01, -2.88962431e-32, ...,
            3.23808374e-32,  7.24097043e-02,  2.68939824e-32],
          [ 9.95511003e-03, -6.79306760e-02, -2.65293265e-32, ...,
           -2.63278996e-32, -7.49826282e-02, -2.73345724e-32]],
 
         [[ 8.81558750e-03,  8.80156606e-02, -3.16908222e-32, ...,
           -3.12782002e-32,  9.79573000e-03,  2.83814295e-32],
          [-1.07979357e-01,  3.85711975e-02, -2.93644366e-32, ...,
            3.03134602e-32,  4.16533835e-02,  2.70892996e-32],
          [-2.08845418e-02, -5.13667613e-02, -2.74312745e-32, ...,
           -2.69124259e-32, -2.12689843e-02, -2.69293854e-32]],
 
         [[ 2.72477046e-03, -4.05968986e-02,  2.88552125e-32, ...,
           -2.98909876e-32, -1.55706555e-02,  2.78963030e-32],
          [-4.87495549e-02, -1.21937245e-02, -3.05091272e-32, ...,
           -3.085

### Training the transfered-model

In [6]:
file="model_resnet18_cam-pitch_transfer-learning_{epoch:02d}.hdf5"
checkpoint =  ModelCheckpoint(os.path.join(checkpoint_dir, file), verbose=1)

NAME = "resnet18_cam-pitch_transfer-learning_{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME), write_graph = True, write_images=True)

new_model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[checkpoint, tensorboard])


Epoch 1/100
6255/6255 [==============================] - 2351s 376ms/step - loss: 0.6120 - acc: 0.8596 - val_loss: 0.4249 - val_acc: 0.8823

Epoch 00001: saving model to C:\Users\hoang\WorkingSpace\TrainingModels\keras\checkpoints\model_resnet18_cam-pitch_transfer-learning_01.hdf5
Epoch 2/100
6255/6255 [==============================] - 2151s 344ms/step - loss: 0.3037 - acc: 0.9398 - val_loss: 0.4314 - val_acc: 0.8849

Epoch 00002: saving model to C:\Users\hoang\WorkingSpace\TrainingModels\keras\checkpoints\model_resnet18_cam-pitch_transfer-learning_02.hdf5
Epoch 3/100
6255/6255 [==============================] - 2137s 342ms/step - loss: 0.2817 - acc: 0.9479 - val_loss: 0.4259 - val_acc: 0.8862

Epoch 00003: saving model to C:\Users\hoang\WorkingSpace\TrainingModels\keras\checkpoints\model_resnet18_cam-pitch_transfer-learning_03.hdf5
Epoch 4/100
6255/6255 [==============================] - 2302s 368ms/step - loss: 0.2633 - acc: 0.9544 - val_loss: 0.5166 - val_acc: 0.8457

Epoch 00004: 

KeyboardInterrupt: 